In [1]:
import pandas as pd
from src.plots import MovementData

In [2]:
leo = pd.read_csv("data/output/Leo/accelerometer2_data.csv")
matthieu = pd.read_csv("data/output/Matthieu/accelerometer2_data.csv")

data = MovementData([leo, matthieu], ["Leo", "Matthieu"])
plots = data.make_all_plots()

In [5]:
plots["Stacked"].show()